# SERLI - AI4Industry - 2025

---
Notebook de Tom BOIREAU
---

## Import des librairies

In [1]:
import cv2
import os
import json
from tqdm import tqdm
import numpy as np

## Variables globales

In [2]:
DATASET_FOLDER = "dataset/"
IMAGE_SIZE = (1280, 720)

## Variables système

## Import des données

In [7]:
image_list = []
label_list = []
if os.path.exists(DATASET_FOLDER):
    print("Dataset folder exists")
    
    # Load the dataset
    for subfolders in os.listdir(DATASET_FOLDER):
        print("Reading subfolder: ", subfolders)
        subfolder_path = os.path.join(DATASET_FOLDER, subfolders)
        
        for file in tqdm(os.listdir(subfolder_path)):
            if file.endswith('.png'):
                image_path = os.path.join(subfolder_path, file)
                file_name = file.split(".")[0]
                json_path = os.path.join(subfolder_path, file_name + ".json")
                
                if os.path.exists(json_path) and os.path.exists(image_path):
                    image = cv2.imread(image_path)
                    image = cv2.resize(image, IMAGE_SIZE)
                    image_list.append(image)
                    
                    with open(json_path) as json_file:
                        data = json.load(json_file)
                        label_list.append(data)
                else:
                    print("json file does not exist for: ", file)
                    
        break
            
else:
    raise Exception("Dataset folder does not exist")

Dataset folder exists
Reading subfolder:  part001


100%|██████████| 2000/2000 [00:29<00:00, 66.92it/s] 


In [13]:
colors_lst = [ 
    [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], 
    [255, 0, 255], [0, 255, 255], [128, 0, 0], [0, 128, 0], 
    [0, 0, 128], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
    [192, 0, 0], [0, 192, 0], [0, 0, 192], [192, 192, 0], 
    [192, 0, 192], [0, 192, 192], [64, 0, 0], [0, 64, 0], 
    [0, 0, 64], [64, 64, 0], [64, 0, 64], [0, 64, 64]
]
# Function to apply watershed segmentation
def apply_watershed(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2)
    
    sure_bg = cv2.dilate(opening, kernel, iterations = 3)
    
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.05 * dist_transform.max(), 255, 0)
    
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    
    markers = cv2.watershed(image, markers)
    
    new_image = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    for i in range(2, len(colors_lst) + 1):
        new_image[markers == i] = colors_lst[i - 1]
        
    new_image[markers == -1] = [255, 255, 255]
    
    return new_image

# Display the segmented images as a video stream
for img in image_list:
    
    cv2.imshow('Segmented Image', apply_watershed(img))
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(100) & 0xFF == ord('s'):
        cv2.imwrite("segmented_image.png", apply_watershed(img))
    

cv2.destroyAllWindows()